## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [1]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

Предварительно преобразуем исходные данные из .csv в .tsv при помощи csv.writer(open('output.txt', 'w+'), delimiter='\t').writerows(csv.reader(open("Task_4_sample_7.csv"))).
В notepad++ убираем пустые строки из файла и загружаем сюда.

In [16]:
with open("output.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}
  print(triples)
  print(score_map)

[['direction', 'combination', '2.25'], ['start', 'match', '4.47'], ['hospital', 'infrastructure', '4.63'], ['century', 'year', '7.59'], ['gem', 'jewel', '8.96'], ['life', 'term', '4.5'], ['development', 'issue', '3.97'], ['opera', 'performance', '6.88'], ['student', 'professor', '6.81'], ['stock', 'life', '0.92'], ['cup', 'article', '2.4'], ['investigation', 'effort', '4.59'], ['Arafat', 'Jackson', '2.5'], ['Mars', 'water', '2.94'], ['computer', 'news', '4.47'], ['street', 'block', '6.88'], ['murder', 'manslaughter', '8.53'], ['cup', 'entity', '2.15'], ['peace', 'atmosphere', '3.69'], ['phone', 'equipment', '7.13'], ['planet', 'sun', '8.02'], ['stock', 'egg', '1.81'], ['money', 'currency', '9.04'], ['dollar', 'yen', '7.78'], ['announcement', 'effort', '2.75'], ['calculation', 'computation', '8.44'], ['sign', 'recess', '2.38'], ['ministry', 'culture', '4.69'], ['media', 'gain', '2.88'], ['opera', 'industry', '2.63'], ['listing', 'proximity', '2.56'], ['psychology', 'science', '6.71'], [

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [20]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: direction-combination
Ground truth score: 2.25

Path: 0.091 
wup: 0.167 
shortest_path: 10.000

Words: start-match
Ground truth score: 4.47

Path: 0.067 
wup: 0.125 
shortest_path: 14.000


Вычисляем для всех пар несколько оценок

In [27]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, ich_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    ich = max([item1.lch_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    ich_list.append(ich)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [25]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(ich_list, true_list)
print("ich Spearman R: %.4f" % coef)


wup  Spearman R: 0.6384
path Spearman R: 0.6142
ich Spearman R: 0.6142


In [29]:
journey = wn.synset('journey.n.01')
print(len(journey.hyponyms()))
print(journey.hyponyms()[0])

16
Synset('commute.n.01')
